In [42]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [43]:
# Load the trained model, scaler, and OHE

# Load the saved model  
model = load_model('model.h5')  
# Load the scaler
with open('scaler.pkl', 'rb') as file:  
    scaler = pickle.load(file)
    
# Load the one-hot encoder
with open('onehot_encoder_geo.pkl', 'rb') as file: 
    onehot_encoder_geo = pickle.load(file)
    
# Load the label encoder gender 
with open('label_encoder_gender.pkl', 'rb') as file:  
    label_encoder_gender = pickle.load(file)

In [44]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',  # Convert to numerical
    'Gender': 'Male',   # Convert to numerical
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [45]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [46]:
# Convert input data to numerical format, including one-hot encoding for Geography and label encoding for gender

# One Hot encoder 'Geography'
geo_encoded = onehot_encoder_geo.transform([input_df['Geography']]).toarray()
# Convert Geography to DataFrame
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))    

g:\Coding\Miniconda\envs\datascience\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [47]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [48]:
# Encode categorical variables 'Gender'
input_df['Gender'] = label_encoder_gender.transform([input_data['Gender']])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [49]:
# Combine one-hot encoded Geography with the rest of the input data
input_df.drop("Geography", axis=1, inplace=True)    # Drop Geography column
input_df = pd.concat([input_df.reset_index(drop=True), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [50]:
# Scale the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [55]:
# Predict the probability of churn
predicted = model.predict(input_scaled)
predicted

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step


array([[0.05944525]], dtype=float32)

In [56]:
predicted_prob = predicted[0][0]  # Probability of churn
predicted_prob

np.float32(0.059445255)

In [ ]:
if predicted_prob > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(predicted_prob * 100))
else:
    print("The customer is not likely to churn with a probability of {:.2f}%".format((1 - predicted_prob) * 100))

AttributeError: 'numpy.float32' object has no attribute '2f'